In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from matplotlib.patches import Ellipse
from scipy.stats.distributions import chi2

In [ ]:
# A function to plot the mean and covariance ellipse of a 2D Gaussian
def plot_covariance_ellipse(mu, cov, std=2, edgecolor='k', facecolor='none', alpha=1):
    plt.scatter(mu[0], mu[1], c='k', s=25, marker='x')

    vals, vecs = np.linalg.eigh(cov)
    order = vals.argsort()[::-1]
    vals = vals[order]
    vecs = vecs[:,order]

    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    width, height = 2 * std * np.sqrt(vals)
    ellip = Ellipse(xy=mu, width=width, height=height, angle=theta, edgecolor=edgecolor, facecolor=facecolor, alpha=alpha)
    plt.gca().add_artist(ellip)

# Exercise 4.1 - State Estimation for mobile robot

Adapted from *J. De Schutter, J. De Geeter, T. Lefebvre, H. Bruyninckx. Kalman Filters: A Tutorial.*

## 1 Introduction
A mobile robot drives in a known environment. The state of the robot at time step $k$ is $x_k = [X_k, Y_k, \theta_k]^T$, where $X_k$ and $Y_k$ are the coordinates of the robot in the world and $\theta_k$ is the orientation of the robot (Figure 1).

<p align="center">
<img src="figs/diff_drive_robot.png" alt="drawing" width="400" centre/>
</p>

The robot has two independently driven front wheelsand two self-aligning back wheels (casters). Encoders at the driving wheels and a gyroscope give information about the tangential velocity $v_k$ and the angular velocity $\omega_k$ of the robot. 

In a first exercise, only the internal sensors are used to estimate the state of the robot. This is called deadreckoning. Making a more accurate estimate is only possible when the robot observes its environment. For example, for an autonomous guided vehicle (AGV), it is of great importance to have an accurate position estimate even after long trajectories. Therefore, people intentionally place markers along the AGV trajectory. The measurements of the markers can reduce the position estimation error. The following exercises consider a robot equipped with an ultrasonic sensor and/or a range finder, and a Kalman Filter (KF) will be used to calculate the robot state estimate. Also consistency of the KF will be checked using the Normalised Innovation Squared (NIS) of the KF.

## 2 Tasks

### 2.1 Deadreckoning

In this exercise, a trajectory of control inputs $[v, \omega]$ are applied, and the state of the robot is estimated based on the initial state estimate $\hat{x}_0$, the initial state estimate covarianve $P_0$, and the process noise $Q$.

Tasks:
* Derive the state transition model $x_k = f(x_{k-1}, u_{k-1})$ using forward Euler discretisation with a time step $\Delta T$ of 0.02 s, and implement it in the code.
* Derive the linearised state transition model $F_k$, and implement it in the code.
* Implement the process update step in the simulation loop.
* Simulate the trajectory of the robot and analyse the results. Do the results make sense? What is the value for the process noise covariance $Q$?
* Change the value of $Q$ to be zero. Run the simulation again. What do you observe? Can you explain the results?
* Simulate the trajectory of the robot for different intial state estimates. What do you observe?
* Observe the diagonal elements of the state estimate covariance matrix $P$. Why are there diagonal elements even though the initial state estimate covariance is diagonal?

### 2.1 Ultrasonic sensor

The robot is equipped with an ultrasonic sensor, which measures twice the perpendicular distance between the robot and the wall. The position of the wall is given by $y=ax + b$. Therefore, the measurement equation is $z_k = h_{\text{US}}(x_k) = \frac{2a}{\sqrt{1+a^2}}X_k - \frac{2}{\sqrt{1+a^2}}Y_k + \frac{2b}{\sqrt{1+a^2}}$. The standard deviation of the measurement noise is $\sigma_{\text{US}}$ = 0.03 m, the measurements are taken every 0.2 s (i.e., one measurement is taken every ten deadreckoning steps).

Tasks:
* Implement the measurement model $h_{\text{US}}(x_k)$ in the code.
* Derive the linearised measurement model $H_k$, and implement it in the code.
* Implement the measurement update step in the simulation loop, and the NIS consistency test.
* Simulate for different wall placements, and watch the state estiamate and its uncertainty. What do you observe? Can you explain the results?
* Look at the results of the NIS consistency test. Does it make sense? What is the value for the measurement noise covariance $R_\text{US}$? What happens if you change the value of $R_\text{US}$?

### 2.2 Range finder

The robot is now equipped with a range finder instead of an ultrasonic sensor. Every 0.5 s, the range finder measures the distance and the angle from the sensor to the beacon.

Tasks:
* Derive the measurement model $z_k = h_\text{RF}(x_k)$, and implement it in the code.
* Derive the linearised measurement model $H_k$, and implement it in the code.
* Implement the measurement update step in the simulation loop, and the NIS consistency test.
* Simulate for different beacon placements, and watch the state estiamate and its uncertainty. What do you observe? Can you explain the results?
* Look at the results of the NIS consistency test. Does it make sense? What is the value for the measurement noise covariance $R_\text{RF}$? What happens if you change the value of $R_\text{RF}$?

### 2.3 Sensor fusion

Use both the ultrasonic sensor and the range finder to estimate the robot position. This is called sensor fusion. How does the result compare to the results when using the individual sensors?

## 3 Code

### State transition function

In [ ]:
# nonlinear state transition function
def f(x, u, timestep):

    X = x[0,0]
    Y = x[1,0]
    theta = x[2,0]
    v = u[0,0]
    omega = u[1,0]

    f = np.zeros(3).reshape(-1, 1)
    
    #########################
    # FILLED IN BY STUDENTS #
    #########################
    f[0,0] = X + v * timestep * np.cos(theta)
    f[1,0] = Y + v * timestep * np.sin(theta)
    f[2,0] = theta + omega * timestep
    #########################

    return f


# liniarize the state transition function
def f_jac(x,u, timestep):
    
    X = x[0,0]
    Y = x[1,0]
    theta = x[2,0]
    v = u[0,0]
    omega = u[1,0]
    F = np.zeros((3,3))
    
    #########################
    # FILLED IN BY STUDENTS #
    #########################
    F[0,0] = 1
    F[0,1] = 0
    F[0,2] = -v * timestep * np.sin(theta)
    F[1,0] = 0
    F[1,1] = 1
    F[1,2] = v * timestep * np.cos(theta)
    F[2,0] = 0
    F[2,1] = 0
    F[2,2] = 1
    #########################

    return F

### Measurement function for ultrasonic sensor

In [ ]:
# position of the wall
wall = np.array([1,3])

# measurement function for the US sensor
def h_US(x):

    X = x[0,0]
    Y = x[1,0]
    theta = x[2,0]
    h = np.zeros((1,1))  

    #########################
    # FILLED IN BY STUDENTS #
    #########################
    denominator = np.sqrt(wall[0]**2 + 1)
    h[0,0] = (2*wall[0]*X)/denominator + (-2*Y)/denominator + (2*wall[1])/denominator
    #########################

    return h

# measurement function jacobian
def h_US_jac(x):

    X = x[0,0]
    Y = x[1,0]
    theta = x[2,0]
    H = np.zeros((1,3))
    
    #########################
    # FILLED IN BY STUDENTS #
    #########################
    denominator = np.sqrt(wall[0]**2 + 1)
    H[0,0] = 2*wall[0]/denominator
    H[0,1] = -2/denominator
    H[0,2] = 0
    #########################
    
    return H

### Measurement function for range finder

In [ ]:
beacon_position = np.array([5,5])

# measurement function for range finder
def h_RF(x):

    X = x[0,0]
    Y = x[1,0]
    theta = x[2,0]
    h = np.zeros((2,1))

    #########################
    # FILLED IN BY STUDENTS #
    #########################
    delta = beacon_position - np.array([X, Y])
    r = np.linalg.norm(delta)
    phi = np.arctan2(delta[1], delta[0]) - theta
    h[0,0] = r
    h[1,0] = phi
    #########################

    return h

# measurement function jacobian
def h_RF_jac(x):

    X = x[0,0]
    Y = x[1,0]
    theta = x[2,0]
    H = np.zeros((2,3))

    #########################
    # FILLED IN BY STUDENTS #
    #########################
    delta = beacon_position - [X, Y]
    q = np.linalg.norm(delta)
    H[0,0] = -delta[0]/q
    H[0,1] = -delta[1]/q
    H[0,2] = 0
    H[1,0] = delta[1]/q**2
    H[1,1] = -delta[0]/q**2
    H[1,2] = -1
    #########################
    
    return H


### Simulation loop

In [ ]:
#############
# Exercises #
#############

# for exercise 2.1
# USE_US = False
# USE_RF = False

# for exercise 2.2
# USE_US = True
# USE_RF = False

# for exercise 2.3
# USE_US = False
# USE_RF = True

# for exercise 2.4
USE_US = True
USE_RF = True

#############

# actual initial state
x_actual = np.array([0, 0, -30*np.pi/180]).reshape(-1, 1)

# estimated initial state
x_hat = np.array([0, 0, -10*np.pi/180]).reshape(-1, 1)
P = np.diag([1**2, 1**2, (20*np.pi/180)**2])
R_RF = np.diag([0.05**2, (0.1*np.pi/180)**2])
R_US = np.array([[0.1**2]])

# process uncertainty (m^2, m^2, rad^2)
Q = np.diag([0.01**2, 0.01**2, (0.1*np.pi/180)**2])
# Q = np.zeros((3, 3))

# time step used in simulation (seconds)
dt = 0.02

log_list = []
NIS_RF = []
NIS_US = []

###################
# SIMULATION LOOP #
###################
for i in range(801):

    log_list.append({
        'x_actual': x_actual,
        'x_hat': x_hat,
        'P': P,
    })

    # control input (velocity, yaw rate)
    u = np.array([0.7, 0.1]).reshape(-1, 1)

    # simulate the robot motion
    process_noise = np.random.multivariate_normal([0, 0, 0], Q).reshape(-1, 1)
    x_actual = f(x_actual, u, dt) + process_noise

    ########################
    # FILLED IN BY STUDENTS#
    ########################
    #  update the estimated state
    x_hat = f(x_hat, u, dt)
    
    # update the state covariance
    F = f_jac(x_hat, u, dt)
    P = F @ P @ F.T + Q
    ########################

    if i % 10 == 0 and USE_US:
        # Update US sensor
        # get the actual measurement
        measurement_noise = np.random.multivariate_normal([0], R_US).reshape(-1, 1)
        z_measured = h_US(x_actual) + measurement_noise

        ########################
        # FILLED IN BY STUDENTS#
        ########################
        # predict what the measurement should be
        z_predicted = h_US(x_hat)
        H = h_US_jac(x_hat)

        # calculate the Kalman gain
        # compute the innovation
        nu = z_measured - z_predicted
        S = H @ P @ H.T + R_US
        K = P @ H.T @ np.linalg.inv(S)

        # calculate the NIS
        NIS_US.append(nu.T @ np.linalg.inv(S) @ nu)


        # update the state estimate
        x_hat = x_hat + K @ nu
        P = (np.eye(3) - K @ H) @ P
        ########################
    
    if i % 25 == 0 and USE_RF:
        # Update RF sensor
        # get the actual measurement
        measurement_noise = np.random.multivariate_normal([0, 0], R_RF).reshape(-1, 1)
        z_measured = h_RF(x_actual) + measurement_noise

        ########################
        # FILLED IN BY STUDENTS#
        ########################
        # predict what the measurement should be
        z_predicted = h_RF(x_hat)
        H = h_RF_jac(x_hat)

        # calculate the Kalman gain
        # compute the innovation
        nu = z_measured - z_predicted
        S = H @ P @ H.T + R_RF
        K = P @ H.T @ np.linalg.inv(S)

        # calculate the NIS
        NIS_RF.append(nu.T @ np.linalg.inv(S) @ nu)

        # update the state estimate
        x_hat = x_hat + K @ nu
        P = (np.eye(3) - K @ H) @ P
        ########################
        

############
# PLOTTING #
############
plt.figure(figsize=(5, 5))
plt.title('Vehicle position and covariance ellipses')

# plot wall
def wall_y(x):
    return wall[0]*x + wall[1]
plt.plot([-10, 15], [wall_y(-10), wall_y(15)], 'k-', linewidth=5)

# plot beacon
plt.plot(beacon_position[0], beacon_position[1], 'bo')

for i in range(len(log_list)):
    log = log_list[i]
    if i % 200 == 0:
        # plot the actual vehicle position in red
        plt.plot(log['x_actual'][0], log['x_actual'][1], 'go')
        # plot the ellipsoid representing the estimated state covariance
        plot_covariance_ellipse(
            (log['x_hat'][0], log['x_hat'][1]), log['P'][0:2, 0:2],
            std=6, facecolor='k', alpha=0.3)

# add a legend
plt.legend(['Wall', 'Beacon', 'Actual position', 'Estimated position', 'Covariance Ellipse'])
plt.ylabel('Y Position (m)')
plt.xlabel('X Position (m)')
plt.ylim(-10, 10)
plt.xlim(-7, 15)
plt.show()

# plot the state estimate and covariance over time
plt.figure(figsize=(10, 10))
plt.subplot(3, 1, 1)

sigma = 6
t_vec = np.arange(0, len(log_list)*dt, dt)
plt.plot(t_vec, [x['x_actual'][0] for x in log_list], 'g')
plt.plot(t_vec, [x['x_hat'][0] for x in log_list], 'k')
plt.fill_between(t_vec, 
                 [x['x_hat'][0,0] - sigma*np.sqrt(x['P'][0, 0]) for x in log_list], 
                 [x['x_hat'][0,0] + sigma*np.sqrt(x['P'][0, 0]) for x in log_list], 
                 color='k', alpha=0.2)
plt.xlim(0, 16) 
plt.xlabel('Time (s)') 
plt.ylabel('X Position (m)')
plt.legend(['Actual', 'Estimated', '3 Sigma'])

plt.subplot(3, 1, 2)
plt.plot(t_vec, [x['x_actual'][1] for x in log_list], 'g')
plt.plot(t_vec, [x['x_hat'][1] for x in log_list], 'k')
plt.fill_between(t_vec, 
                 [x['x_hat'][1,0] - sigma*np.sqrt(x['P'][1, 1]) for x in log_list], 
                 [x['x_hat'][1,0] + sigma*np.sqrt(x['P'][1, 1]) for x in log_list], 
                 color='k', alpha=0.2)
plt.xlim(0, 16)
plt.xlabel('Time (s)') 
plt.ylabel('Y Position (m)')
plt.legend(['Actual', 'Estimated', '3 Sigma'])

plt.subplot(3, 1, 3)
plt.plot(t_vec, [x['x_actual'][2] for x in log_list], 'g')
plt.plot(t_vec, [x['x_hat'][2] for x in log_list], 'k')
plt.fill_between(t_vec,
                 [x['x_hat'][2,0] - sigma*np.sqrt(x['P'][2, 2]) for x in log_list], 
                 [x['x_hat'][2,0] + sigma*np.sqrt(x['P'][2, 2]) for x in log_list], 
                 color='k', alpha=0.2)
plt.xlim(0, 16)
plt.xlabel('Time (s)') 
plt.ylabel('Orientation (rad)')
plt.legend(['Actual', 'Estimated', '3 Sigma'])
plt.show()

if len(NIS_US) != 0:
    # plot the NIS over time for the ultrasonic sensor
    plt.figure(figsize=(10, 5))
    plt.scatter(t_vec[::10], NIS_US, color='k', s=10, marker='x')
    plt.axhline(chi2.ppf(0.975, df=1), color='r', linestyle='--')
    plt.xlabel('Time (s)')
    plt.ylabel('Ultrasonic NIS')
    plt.legend(['NIS', 'Threshold'])
    plt.show()

if len(NIS_RF) != 0:
    # plot the NIS over time for the range finder
    plt.figure(figsize=(10, 5))
    plt.scatter(t_vec[::25], NIS_RF, color='k', s=10, marker='x')
    plt.axhline(chi2.ppf(0.975, df=2), color='r', linestyle='--')
    plt.xlabel('Time (s)')
    plt.ylabel('Range finder NIS')
    plt.legend(['NIS', 'Threshold'])
    plt.show()

